In [1]:
import sys
import os

# Absolute path to your module's directory
module_path = os.path.abspath('/localscratch/zippoema/git/HPA/src/')
if module_path not in sys.path:
    sys.path.insert(0, module_path)

import hpa

In [20]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import gsd.hoomd 
from sklearn import cluster as cl


In [3]:
plt.style.use('default')
plt.style.use('/localscratch/zippoema/md_plots/rates_plot.mplstyle')

In [4]:
path_200 = '/localscratch/zippoema/project_ness/simulations/200-tdp43_CAL3/'
path_ss = '/localscratch/zippoema/project_ness/simulations/search_steadystate/'
path_work = '/localscratch/zippoema/working_dir/'

## Compute s(c0_{in}) and s(c0_{out})

In [120]:
input_file = path_work+'no_driving_slab/sim_eq_slab_200-tdp43-lcd_2-ck1d-open_10-pp3/sim1_eq_slab_200-tdp43-lcd_2-com-ck1d-open_10-com-pp3_CAL3_'
tr = gsd.hoomd.open(input_file+'dump_centered.gsd', 'rb')
c = np.loadtxt(input_file+'contacts.txt')

/localscratch/zippoema/lib/python_envs/hpa_env/lib/python3.11/site-packages/gsd/hoomd.py:1125: FutureWarning: The 'rb' mode is deprecated, use 'r'
  gsdfileobj = fl.open(name=str(name),


In [121]:
type_of_contact=1
c_p = c[c[:, 2] == type_of_contact]
type_of_contact=-1
c_dp = c[c[:, 2] == type_of_contact]
times_p = np.rint(c_p[:,0]/200000).astype(int)
times_dp = np.rint(c_dp[:,0]/200000).astype(int)

In [122]:
ser_i = np.array([266, 273, 292, 305, 317, 332, 333, 342, 347, 350, 369, 373, 375, 377, 379, 387, 389, 393, 395, 403, 404, 407, 409, 410])-260-1 
ser_i = np.array([ ser_i+154*j for j in range(200) ]).flatten()

eps=1.0
min_sample=2
count_p = 0
in_out = np.zeros((len(tr)+1,len(ser_i)))
frame = gsd.hoomd.open(input_file+'start_centered.gsd', 'r')[0]
positions = frame.particles.position
type_ids = frame.particles.typeid[:30800]
db = cl.DBSCAN(eps=eps, min_samples=min_sample).fit(positions)
labels = db.labels_
values, counts = np.unique(labels[:30800], return_counts=True)
condensate_idx = values[np.argmax(counts)]
in_out[0] = labels[ser_i]==condensate_idx
for i, frame in enumerate(tqdm(tr)):
    positions = frame.particles.position
    type_ids = frame.particles.typeid[:30800]
    db = cl.DBSCAN(eps=eps, min_samples=min_sample).fit(positions)
    labels = db.labels_
    values, counts = np.unique(labels[:30800], return_counts=True)
    condensate_idx = values[np.argmax(counts)]
    in_out[i+1] = labels[ser_i]==condensate_idx

100%|█████████████████████████████████████████████████████████████████| 1250/1250 [02:34<00:00,  8.10it/s]


In [128]:
count_p = 0
for i, tt in enumerate(tqdm(times_p)):
    ser_index = np.where(ser_i==c_p[i,1])[0][0]
    count_p += int(in_out[tt,ser_index])
    if not in_out[tt,ser_index]:
        print(i, tt, c_p[i,1], c_p[i,5])
        

 85%|█████████████████████████████████████████████████▉         | 43471/51318 [00:00<00:00, 222947.08it/s]

31 0 16134.0 0.0
32 0 23670.0 10.0
33 0 16134.0 0.0
34 0 23606.0 10.0
35 1 16142.0 0.0
36 1 16132.0 0.0
37 1 16134.0 0.0
38 1 23651.0 10.0
40 1 13694.0 1.0
41 1 13694.0 1.0
42 1 13694.0 1.0
43 1 13695.0 1.0
44 1 8464.0 2.0
45 1 23593.0 3.0
67 2 20288.0 3.0
68 2 20288.0 3.0
69 2 20288.0 3.0
141 2 30690.0 2.0
209 3 16144.0 9.0
213 3 16148.0 9.0
215 3 16105.0 9.0
217 3 16150.0 9.0
219 3 16148.0 9.0
224 3 16097.0 9.0
229 3 16097.0 9.0
232 3 16097.0 9.0
235 3 16105.0 9.0
236 3 16088.0 9.0
237 3 16088.0 9.0
251 3 16144.0 9.0
254 3 5693.0 5.0
257 3 16102.0 9.0
258 3 16105.0 9.0
260 3 16105.0 9.0
266 3 16047.0 9.0
268 3 16047.0 9.0
292 3 6768.0 5.0
306 4 2520.0 10.0
309 4 2580.0 10.0
310 4 2578.0 10.0
311 4 2592.0 10.0
312 4 2613.0 10.0
339 4 210.0 6.0
341 4 210.0 6.0
396 4 286.0 0.0
399 4 288.0 0.0
402 4 288.0 0.0
403 4 26776.0 3.0
404 4 286.0 0.0
405 4 26774.0 3.0
406 4 286.0 0.0
407 4 26770.0 3.0
409 4 288.0 0.0
413 5 210.0 0.0
414 5 26758.0 3.0
415 5 210.0 0.0
416 5 26758.0 3.0
417 5 210.0

100%|███████████████████████████████████████████████████████████| 51318/51318 [00:00<00:00, 222191.21it/s]

43483 1139 27955.0 5.0
43484 1139 27960.0 5.0
43486 1139 25091.0 7.0
43487 1139 6781.0 6.0
43491 1139 6857.0 6.0
43509 1140 6910.0 0.0
43510 1140 6902.0 0.0
43511 1140 27990.0 5.0
43512 1140 6781.0 9.0
43515 1140 27905.0 5.0
43521 1140 24404.0 9.0
43523 1140 24403.0 9.0
43524 1140 24404.0 9.0
43525 1140 24404.0 9.0
43526 1140 28023.0 5.0
43527 1140 28022.0 5.0
43528 1140 28016.0 5.0
43539 1140 30795.0 5.0
43540 1140 30792.0 5.0
43543 1140 30778.0 5.0
43544 1140 30774.0 5.0
43545 1140 30772.0 5.0
43546 1140 30735.0 5.0
43555 1141 30754.0 5.0
43556 1141 30754.0 5.0
43558 1141 30754.0 5.0
43559 1141 30735.0 5.0
43560 1141 30658.0 5.0
43561 1141 30658.0 5.0
43562 1141 30658.0 5.0
43563 1141 30658.0 5.0
43564 1141 30658.0 5.0
43565 1141 30690.0 5.0
43566 1141 30690.0 5.0
43567 1141 30677.0 5.0
43568 1141 30690.0 5.0
43569 1141 30764.0 2.0
43570 1141 30762.0 2.0
43576 1142 30789.0 7.0
43577 1142 30789.0 7.0
43578 1142 28016.0 4.0
43579 1142 28017.0 4.0
43580 1143 25730.0 10.0
43583 1143 2578

In [124]:
count_dp = 0
for i, tt in enumerate(tqdm(times_dp)):
    ser_index = np.where(ser_i==c_dp[i,1])[0][0]
    count_dp += int(in_out[tt,ser_index])


100%|███████████████████████████████████████████████████████████| 51298/51298 [00:00<00:00, 315728.36it/s]


In [125]:
count_p, count_dp

(47031, 47035)

In [114]:
np.where(ser_i==28983)[0][0]

4514

In [115]:
ser_i[4514]

28983

## Compute diffusion coefficient in dilute